In [3]:
import pandas as pd
import numpy as np
import re
import os
import string


In [5]:
def read_file(directory = "CSE508_Winter2023_Dataset_Processed"):
    data = {}
    for filename in os.listdir(directory):    
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as f:
            file_contents = f.read()
        data[filename] = file_contents
    return data

data = read_file()
print("File count = ",len(data))

keys = list(data.keys())

s = "";
df = []
for key in data:
    df.append(data[key])
    


File count =  1400


In [9]:
from collections import defaultdict

def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    
    
    for doc_id, document in enumerate(documents):
        string1 = document.split()
        for word in range(len(string1)-1):
            bi_string = string1[word:word+2]
            bi = " ".join(bi_string)
            inverted_index[bi].append(doc_id)
    
    return inverted_index


defaultdict(list,
            {'experimental investigation': [0,
              28,
              83,
              188,
              250,
              371,
              371,
              422,
              441,
              441,
              496,
              504,
              521,
              568,
              634,
              635,
              661,
              693,
              712,
              765,
              771,
              800,
              815,
              835,
              843,
              857,
              886,
              995,
              1038,
              1073,
              1091,
              1096,
              1097,
              1155,
              1158,
              1160,
              1204,
              1219,
              1224,
              1226,
              1229,
              1337,
              1363],
             'investigation of': [0,
              8,
              18,
              28,
              29,
              4

In [11]:
import pickle
from collections import defaultdict

inverted_index = create_inverted_index(df)

def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)
        
def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
        
    return inverted_index